# Host Filtering Assessment

Filtering the barley genome from NT metagenomic reads was observed to bias taxonomic classifications, with a reduction in Firmicutes. Let's try to work out why...

## Barley Genome

We can use the Morex V3 assembly which includes PacBio HiFi reads, and so is less likely to have bacterial contaminants in the assembly than Morex V2 which was used for the previous analysis. 

In [11]:
import urllib3
import shutil
import subprocess
import os
import re
from glob import glob
import pysam
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Bio import SeqIO
import sys
import gzip
import shutil

samples={'bulk':['2000','2001','2002'], 'elite':['2023','2024','2025']}


In [ ]:
os.mkdir('reference')
http = urllib3.PoolManager()

fasta_url='https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/b6e6a2e5-2746-4522-8465-019c8f56df7f/1/DOWNLOAD'
annot_url='https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/5d16cc17-c37f-417f-855d-c5e72c721f6c/1/DOWNLOAD'
fasta_path='reference/Barley_MorexV3_psuedomolecules.fasta'
annot_path='reference/Barley_MorexV3_psuedomolecules.gff3'
    
print('Downloading {}'.format(fasta_url))
with http.request('GET',fasta_url, preload_content=False) as resp, open(fasta_path, 'wb') as out_file:
    shutil.copyfileobj(resp, out_file)
r.release_conn()

print('Downloading {}'.format(annot_url))
with http.request('GET',annot_url, preload_content=False) as resp, open(annot_path, 'wb') as out_file:
    shutil.copyfileobj(resp, out_file)
r.release_conn()

Now we can build a BWA index...

In [ ]:
os.mkdir('bwa')
os.mkdir('logs')
bwa_path='bwa/MorexV3'
os.symlink('../'+fasta_path,bwa_path)

process=subprocess.Popen(['bwa','index',bwa_path],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE)
stdout,stderr = process.communicate()

with open('logs/bwa_morex_v3_index.out','w') as stdout_file:
    stdout_file.write(stdout.decode('UTF-8'))

with open('logs/bwa_morex_v3_index.err','w') as stderr_file:
    stderr_file.write(stderr.decode('UTF-8'))

...and symlink in the original QC'd fastq files...

In [ ]:
files=glob('../GO_enrichment/read_qc/trim_galore/*gz')
if not os.path.exists('fastq'):
    os.mkdir('fastq')

fq_re=re.compile(r'([0-9]{4}_[12])_val_[12].fq.gz')
for file in files:
    m=re.search(fq_re,file)
    os.symlink('../'+file,'fastq/{}.fq.gz'.format(m.group(1)))

In [ ]:
%%bash
cat bin/align_and_filter.sh
#qsub bin/align_and_filter.sh

### Mapping Results

In [11]:
tot_re=re.compile(r'^([0-9]+) \+ [0-9]+ in total')
map_re=re.compile(r'([0-9]+) \+ [0-9]+ mapped')

labels=[]
totals=[]
mapped=[]

for sample in samples.keys():
    for rep in samples[sample]:   
        print(rep)
        labels.append(rep)
        flagstat=pysam.flagstat("bwa_alignments/{}.sorted.bam".format(rep))
        m=re.search(tot_re,flagstat)
        totals.append(m.group(1))
    
        m=re.search(map_re,flagstat)
        mapped.append(m.group(1))
        


2000
2001
2002
2023
2024
2025


IOStream.flush timed out
IOStream.flush timed out


In [ ]:

totals=[int(x) for x in totals]
mapped=[int(x) for x in mapped]
percents=[]
for i in range(6):
    percents.append((mapped[i]/totals[i])*100)

x=np.arange(len(labels))
width=0.35

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

fig, (ax1,ax2) = plt.subplots(1,2)
rects1 = ax1.bar(x - width/2, totals, width, color=(1, 0.607, 0.314, 1),label='Total Reads')
rects2 = ax1.bar(x + width/2, mapped, width, label='Mapped Reads')

ax1.set_ylabel('Reads')
ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.ticklabel_format(axis='y',style='sci')

rects3 = ax2.bar(x,percents,width*2)
ax2.set_xticks(x)
ax2.set_xticklabels(labels)
ax2.set_ylabel('Mapped Reads (%)')


fig.suptitle('Bulk and Elite Read Mapping')
fig.legend(loc='upper center',bbox_to_anchor=(0.5,-0.01),ncol=2)
fig.tight_layout()

plt.show()

## Kraken

We can now classify both the filtered and unfiltered reads with Kraken2 and the kraken2_pluspfp database.

In [14]:
%%bash
cat bin/kraken.sh

#!/bin/env bash

#$ -pe smp 24
#$ -mods l_hard mfree 96G
#$ -j y
#$ -o logs/$JOB_NAME.$JOB_ID.$TASK_ID
#$ -cwd
#$ -t 1-12

set -e

# Runs kraken2 against paired fastq reads, where read filenames start with the sample followed by a '_'.

# This script is designed to be run as an array job on an SGE cluster 
# which has local disk storage available on execution nodesaccessed via $TMPDIR, 

usage="Usage: $0 -i /path/to/input/fastq/directory -o /path/to/output/directory -d path/to/kraken_db" 

while getopts "i:o:d:" opt; do
  case "${opt}" in
	i )
		in_dir=${OPTARG}
		;;
	o )
	  out_dir=${OPTARG}
	  ;;
	d )
		db=${OPTARG}
	  ;;
	\? )
		echo ${usage}
	  ;;
	: )
		echo "Invalid option: $OPTARG requires an argument" 1>&2
	;;
  esac
done

if [[ ! -e "${in_dir}" ]]; then
	echo "Specificed input directory (${indir}) does not exist"
	exit
fi

if [[ ! -e "${db}" ]]; then
	echo "Specificed kraken database (${db}) does not exist"
	exit
fi

if [[ -z "${SGE_TASK_ID}" ]]; then
  echo "This script shoul

In [ ]:
%%bash
qsub bin/kraken.sh -i fastq -o kraken/nt_unfiltered -d /cluster/db/jabbott/training_data/kraken2_pluspfp_20210127/
qsub bin/kraken.sh -i filtered_fastq -o kraken/nt_morex3 -d /cluster/db/jabbott/training_data/kraken2_pluspfp_20210127/

### Create Krona outputs

In [ ]:
%%bash
mkdir -p kraken/krona
cd kraken/nt_unfiltered
ktImportTaxonomy -i -m 3 -t 5 -c 2000.report.txt 2001.report.txt  2002.report.txt -o ../krona/nt_unfiltered_bulk.html
ktImportTaxonomy -i -m 3 -t 5 -c 2023.report.txt 2024.report.txt  2025.report.txt -o ../krona/nt_unfiltered_elite.html

cd ../nt_morex3
ktImportTaxonomy -i -m 3 -t 5 -c 2000.report.txt 2001.report.txt  2002.report.txt -o ../krona/nt_morex3_bulk.html
ktImportTaxonomy -i -m 3 -t 5 -c 2023.report.txt 2024.report.txt  2025.report.txt -o ../krona/nt_morex3_elite.html


__Results__

* [Unfilted Bulk](kraken/krona/nt_unfiltered_bulk.html)
* [Unfiltered Elite](kraken/krona/nt_unfiltered_elite.html)
* [Morex3 filtered Bulk](kraken/krona/nt_morex3_bulk.html)
* [Morex3 filtered Elite](kraken/krona/nt_morex3_elite.html)

## DESeq2 Analysis

DESeq2 was used to determine any statistically significant changes to the community composition introduced by the filtering process, based upon the kraken2 results. See [host_filter.Rmd](host_filter.Rmd) for the code...

This only looked at bulk and elite samples, since elite is Morex then this should be optimally filtered by alignment. Bulk as expected mainly shows changes in plant species.

In [2]:
bulk_deseq=pd.read_csv('deseq2/bulk.txt',sep="\t")
bulk_deseq

,taxa,taxid,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pre.2000,pre.2001,pre.2002,post.2000,post.2001,post.2002
0,Medicago,3877,1.183987e+05,-2.397232,0.339698,-7.056959,1.701860e-12,1.825099e-09,73868,52666,92400,105471,74966,128028
1,Triticum,4564,7.090361e+06,-2.305467,0.331101,-6.963023,3.330473e-12,1.825099e-09,4650548,3379867,5795182,6273404,4520301,7499573
2,Geosporobacter,390805,1.006955e+02,-1.842569,0.296498,-6.214440,5.150787e-10,1.411316e-07,100,73,85,107,53,89
3,Brachypodium,15367,7.370273e+03,-1.809906,0.314330,-5.757985,8.512398e-09,1.865918e-06,6554,4821,7625,6308,4305,7216
4,Mogibacterium,86331,1.784931e+02,-1.458128,0.325623,-4.477966,7.535773e-06,6.353237e-04,197,143,215,180,85,149
5,Setaria,4554,4.825621e+03,-1.424191,0.288600,-4.934821,8.022439e-07,1.099074e-04,5364,4045,5840,3829,2749,4353
6,Zea,4575,8.038816e+03,-1.372550,0.291772,-4.704185,2.548825e-06,3.103903e-04,9249,6876,9962,6485,4396,7177
7,Panicum,4539,4.302652e+03,-1.356234,0.271575,-4.993949,5.915698e-07,9.262293e-05,5028,3773,5286,3561,2409,3694
8,Homo,9605,1.823806e+05,-1.244964,0.199390,-6.243862,4.268968e-10,1.411316e-07,209346,210294,208995,132190,131857,135838
9,Sorghum,4557,4.014148e+03,-1.236504,0.273783,-4.516362,6.291111e-06,6.152143e-04,4918,3834,5179,3082,2233,3453


In [3]:
elite_deseq=pd.read_csv('deseq2/elite.txt',sep="\t")
elite_deseq

,taxa,taxid,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pre.2023,pre.2024,pre.2025,post.2023,post.2024,post.2025
0,Blastochloris,59282,4983.233017,-0.398737,0.121450,-3.283141,1.026572e-03,0.039921,7740,7091,7949,3194,2859,3680
1,Polymorphum,991903,1685.836432,-0.388073,0.098087,-3.956401,7.608749e-05,0.016623,2554,2532,2657,1056,1016,1208
2,Alicycliphilus,201096,1367.673883,-0.387803,0.108749,-3.566037,3.624201e-04,0.028359,1877,2280,2136,813,869,981
3,Thermobispora,147067,1741.908430,-0.374820,0.106888,-3.506654,4.537793e-04,0.028374,2771,2687,2577,1096,1095,1178
4,Chelatococcus,28209,4136.936104,-0.369665,0.116961,-3.160584,1.574532e-03,0.047427,6549,6053,6523,2689,2390,2917
5,Starkeya,152053,5597.912921,-0.357750,0.113382,-3.155261,1.603548e-03,0.047427,8845,8290,8868,3603,3217,3965
6,Pannonibacter,227873,1326.701856,-0.357417,0.109139,-3.274871,1.057102e-03,0.039921,2084,2030,2049,852,822,876
7,Xanthobacter,279,2080.650012,-0.355472,0.106417,-3.340374,8.366578e-04,0.038606,3227,3148,3302,1316,1208,1484
8,Azorhizobium,6,1985.394327,-0.342947,0.100676,-3.406454,6.581268e-04,0.032704,3070,3070,3140,1253,1164,1391
9,Pontivivens,1844015,1066.529393,-0.339530,0.097943,-3.466604,5.270772e-04,0.028374,1600,1659,1736,639,650,757


Elite shows reductions in a small number of baterial genera, all of which have decent numbers of counts associated with them. Interestingly, Triticum and Nicotiana and increased, presumably as a result of the removal of a considerable amount of sequences effectively enriching them. The Triticum in question is emmer wheat. 

We'll just focus on elite for now, since this is where we seem to see a problem.

### Collect taxids...

We have taxids at the genus level for the species of interest, but some classifications may be at a lower level, so we need to get the full set of taxids. The output of kraken-inspect which is included with the database contains all the details of the included taxa, so we can trawl this to find ids of relevance.

In [4]:
genus_ids=elite_deseq['taxid'].tolist()
genus_ids=list(map(str, genus_ids))
tax_ids=dict()

The list of genus taxids should be screened to select only the bacteria. Let's reuse our existing taxonomy database code.

In [5]:
sys.path.append('bin') 
from taxonomy import download_taxdb,parse_taxonomy

if not os.path.isdir('taxonomy'):
    os.mkdir('taxonomy')
    download_taxdb('taxonomy')
    
nodes,names=parse_taxonomy('taxonomy')

In [6]:
def is_bacteria(taxid,nodes):
    if int(taxid) in nodes:
        node_data=nodes[int(taxid)]
        rank=node_data['rank']
        while node_data['rank'] != 'superkingdom':
            node_data=nodes[node_data['parent']]
        if node_data['name']=='Bacteria':
            return(True)
        else:
            return(False)

In [7]:
bacteria_ids=list()
for id in genus_ids:
    if is_bacteria(id,nodes)==True:
        bacteria_ids.append(id)

The kraken-inspect output is a tab-delimited file with the taxonomy ids in column 5, while column 4 includes a coded taxon level i.e G, S, S2:

```
3.53  638345314       7784546 F5      163735             Phaseoleae
1.41  254736019       16589388        G       3913         Vigna
0.49  88769661        88769661        S       3917           Vigna unguiculata
0.42  75549874        0       S       157791                 Vigna radiata
0.42  75549874        75549874        S1      3916           Vigna radiata var. radiata
0.41  73827096        73827096        S       3914           Vigna angularis
1.02  184902669       0       G       3846                 Glycine
  ```
  
All the species/subspecies ranks immediately follow the genus, so we can scan until we find the taxid of interest then consume rows with an 'S?' code in column 4 to gain the additional tax ids.

In [8]:
def find_ids(taxid):
    ids=list()
    with open('../training_data/kraken2_pluspfp_20210127/inspect.txt', 'r') as handle:
        for line in handle:
            # skip comments
            if re.search('^#',line):
                continue
                
            fields=line.strip().split("\t")
            if fields[4]==taxid:
                print("Found ",fields[5].lstrip())
                ids.append(taxid)
                genus=fields[5].lstrip()
                for line in handle:
                    fields=line.strip().split("\t")
                    if re.search('^(S|G[0-9])',fields[3]):
                        print('Found a species: ',fields[5].lstrip())
                        ids.append(fields[4])
                    else:
                        return(genus,ids)
                    

In [9]:
for id in bacteria_ids:
    (genus,ids)=find_ids(id)
    tax_ids[genus]=ids

Found  Blastochloris
Found a species:  Blastochloris tepida
Found a species:  Blastochloris viridis
Found  Polymorphum
Found a species:  Polymorphum gilvum
Found a species:  Polymorphum gilvum SL003B-26A1
Found  Alicycliphilus
Found a species:  Alicycliphilus denitrificans
Found a species:  Alicycliphilus denitrificans K601
Found a species:  Alicycliphilus denitrificans BC
Found  Thermobispora
Found a species:  Thermobispora bispora
Found a species:  Thermobispora bispora DSM 43833
Found  Chelatococcus
Found a species:  unclassified Chelatococcus
Found a species:  Chelatococcus sp. CO-6
Found a species:  Chelatococcus daeguensis
Found  Starkeya
Found a species:  unclassified Starkeya
Found a species:  Starkeya sp. ORNL1
Found a species:  Starkeya novella
Found a species:  Starkeya novella DSM 506
Found  Pannonibacter
Found a species:  Pannonibacter phragmitetus
Found a species:  Pannonibacter phragmitetus BB
Found  Xanthobacter
Found a species:  Xanthobacter autotrophicus
Found a speci

### Subset Reads

Now we know which tax ids we are looking for we can use the Kraken classifications to separate out the reads identified as belonging to each species of interest. This will require parsing the kraken output files to pull out the read ids which are assigned to the required tax ids, then parsing the fastq files to separate these out.

Do not expect this to be quick...or pretty...

In [12]:
taxid_re=re.compile('taxid ([0-9]+)')

if not os.path.isdir('per_genus_fastq'):
    os.mkdir('per_genus_fastq')
    
for sample in samples['elite']:
    read_ids={} # dict to store read ids per genus
    print('Parsing {} kraken output...'.format(sample))
    with open('kraken/nt_unfiltered/{}.txt'.format(sample)) as handle:
        for line in handle:
            fields=line.strip().split("\t")
            m=re.search(taxid_re,fields[2])
            if m:
                read_tax_id=m.group(1)
                for genus in tax_ids.keys():
                    if read_tax_id in tax_ids[genus]:
                        if genus in read_ids.keys():
                            read_ids[genus].append(fields[1])                            
                        else:
                            read_ids[genus]=[fields[1]]        

    # uncompress reads to $TMPDIR and index for random access...
    for read in ('1','2'):
        with gzip.open('fastq/{}_{}.fq.gz'.format(sample,read),'rb') as f_in:
            with open('{}/{}_{}.fq'.format(os.environ['TMPDIR'],sample,read), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
                
    print('Reading {} F reads'.format(sample))
    f_reads = SeqIO.index('{}/{}_1.fq'.format(os.environ['TMPDIR'],sample), "fastq")
    print('Reading {} R reads'.format(sample))
    r_reads = SeqIO.index('{}/{}_2.fq'.format(os.environ['TMPDIR'],sample), "fastq")

    # Collect reads for each genus and write to separate fastq files...
    for genus in read_ids.keys():   
        genus_f_reads=[]
        genus_r_reads=[]
        
        for read in read_ids[genus]:
            try:
                f=f_reads[read]
            except KeyError:
                print('{} f read not found'.format(read))
            
            try:
                r=r_reads[read]
            except KeyError:
                print('{} r read not found'.format(read))
            
            if f and r:
                genus_f_reads.append(f)
                genus_r_reads.append(r)
    
        with open('per_genus_fastq/{}_1.fq'.format(genus),'a') as out:
            SeqIO.write(genus_f_reads, out, 'fastq')
        with open('per_genus_fastq/{}_2.fq'.format(genus),'a') as out:
            SeqIO.write(genus_r_reads, out, 'fastq')

Parsing 2023 kraken output...
Reading 2023 F reads
Reading 2023 R reads
Parsing 2024 kraken output...
Reading 2024 F reads
Reading 2024 R reads
Parsing 2025 kraken output...
Reading 2025 F reads
Reading 2025 R reads


In [13]:
# Now compress the finished per-genus fastq files...
for genus in read_ids.keys():
    for read in ('1','2'):
        with open('per_genus_fastq/{}_{}.fq'.format(genus,read), 'rb') as f_in:
            with gzip.open('per_genus_fastq/{}_{}.fq.gz'.format(genus,read), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove('per_genus_fastq/{}_{}.fq'.format(genus,read))